#### Observe that we can get news of a day with the link format:
https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=3PzzzV/search#result

In [467]:
import sys
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [575]:
def get_paper_link(paper,tr_len):
    paper_link=''
    for tr_index in range(1,tr_len):
        link_node=paper.select('#format0_disparea > tbody > tr:nth-of-type('+str(tr_index)+')') # 每列
        row=link_node[0].find_all("th", class_="std1")
        if '本論文永久網址1:' in row[0].string:
                row1=paper[0].select("#fe_text1")
                paper_link=row1[0].get('value')
                break
    #     link_node=paper.select_one('#fe_text1')
    #     paper_link=link_node.get('value')
    return paper_link

def get_chinese_title(paper):
    link_node=paper.select_one('#format0_disparea > tbody > tr:nth-of-type(4) > td')
    chinese_title=link_node.string
    return chinese_title

def get_english_title(paper):
    link_node=paper.select_one('#format0_disparea > tbody > tr:nth-of-type(5) > td')
    english_title=link_node.string
    return english_title

def get_paper_year(paper):
    link_node=paper.select_one('#format0_disparea > tbody > tr:nth-of-type(13) > td')
    paper_year=link_node.string
    return paper_year

def get_chinese_keyword(paper):
    link_node=paper.select('#format0_disparea > tbody > tr:nth-of-type(16) > td ')
    keywords=""
    for ele in link_node:
        keywords=keywords+ele.get_text()
    return keywords

def get_english_keyword(paper):
    link_node=paper.select('#format0_disparea > tbody > tr:nth-of-type(17) > td')
    keywords=""
    for ele in link_node:
        keywords=keywords+ele.get_text()
    return keywords

def get_paper_abstract(browser,paper):
    browser.find_element_by_xpath('//*[@id="gs32_levelrecord"]/ul/li[2]/a').click()
    link_node=paper.select_one('#format0_disparea > tbody > tr > td.stdncl2 > div')
    abstract=link_node.get_text()
    abstract=abstract.strip()
    abstract=abstract.replace('               ','').replace('\n','')
    return abstract

def get_paper_info(browser,paper): # link_node以解析過 與link不同
    chinese_title=get_chinese_title(paper)
    english_title=get_english_title(paper)
    chinese_keyword=get_chinese_keyword(paper)
    english_keyword=get_english_keyword(paper)
    paper_year=get_paper_year(paper)
    link=get_paper_link(paper)
    abstract=get_paper_abstract(browser,paper)
    
    info = {'chinese_title' : chinese_title,
            'english_title': english_title,
            'chinese_keyword':chinese_keyword,
            'english_keyword':english_keyword,
            'paper_year':paper_year,
            'link' : link,
            'abstract': abstract}
    return info

In [590]:
browser = webdriver.Chrome('./chromedriver')
browser.get('https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=9P1K1H/webmge?mode=basic')
element=browser.find_element_by_id("ysearchinput0")
element.send_keys("機器學習", Keys.ARROW_DOWN)
browser.find_element_by_name("gs32search").click()
# 變化
paper_page=browser.find_element_by_class_name("slink")
paper_page.click()# 點1
data=list()
for i in range(0,20):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    link_node=soup.select('#format0_disparea > tbody')
    tr_len=len(link_node[0])
    url=''
    chinese_title=''
    english_title=''
    year=''
    chinese_keyword=''
    english_keyword=''
    for tr_index in range(1,tr_len):
        link_node=soup.select('#format0_disparea > tbody > tr:nth-of-type('+str(tr_index)+')') # 每列
        row=link_node[0].find_all("th", class_="std1")
        if '本論文永久網址:' in row[0].string:
            row1=link_node[0].select("#fe_text1")
            link=row1[0].get('value')
            #print(url)
        elif row[0].string=='論文名稱:':
            back=link_node[0].select("td.std2")
            chinese_title=back[0].get_text()
            #print(chinese_title)
        elif row[0].string=='論文名稱(外文):':
            back=link_node[0].select("td.std2")
            english_title=back[0].get_text()
            #print(english_title)
        elif row[0].string=='論文出版年:':
            back=link_node[0].select("td.std2")
            year=back[0].get_text()
            #print(year)
        elif row[0].string=='中文關鍵詞:':
            back=link_node[0].select("td.std2")
            chinese_keyword=back[0].get_text()
            #print(chinese_keyword)
        elif row[0].string=='外文關鍵詞:':
            back=link_node[0].select("td.std2")
            english_keyword=back[0].get_text()
            #print(english_keyword)
    abstract=get_paper_abstract(browser,soup)
    info = {'chinese_title' : chinese_title,
            'english_title': english_title,
            'chinese_keyword':chinese_keyword,
            'english_keyword':english_keyword,
            'paper_year':year,
            'link' : link,
            'abstract':abstract
           }
    data.append(info)
    print("paper-"+str(i+1)+" finish!")
    next_paper=browser.find_element_by_css_selector('#bodyid > form:nth-child(16) > div > table > tbody > tr:nth-child(1) > td.etds_mainct > table > tbody > tr:nth-child(6) > td > div.cont_l2 > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td:nth-child(4) > input[type="image"]')
    next_paper.click()
browser.close()

paper-1 finish!
paper-2 finish!
paper-3 finish!
paper-4 finish!
paper-5 finish!
paper-6 finish!
paper-7 finish!
paper-8 finish!
paper-9 finish!
paper-10 finish!
paper-11 finish!
paper-12 finish!
paper-13 finish!
paper-14 finish!
paper-15 finish!
paper-16 finish!
paper-17 finish!
paper-18 finish!
paper-19 finish!
paper-20 finish!


In [592]:
import pandas as pd
pd.DataFrame(data)[['chinese_title', 'english_title', 'chinese_keyword', 'english_keyword', 'paper_year','link','abstract']].head(19)

,chinese_title,english_title,chinese_keyword,english_keyword,paper_year,link,abstract
0,利用通用型機器學習模式發展特定的機器學習範式,Development of Specific Machine Learning Parad...,人工智慧、機器學習,Artificial Intelligence、Maching Learning,1993,https://hdl.handle.net/11296/7wkt54,本論文是探討如何利用通用型機器學習模式來建構各種特定的機器學習範式。通用型模式訂出五個在機器...
1,應用時間序列與機器學習預測股票最佳化投資組合,Application of time series and machine learnin...,時間序列、模糊理論、遺傳演算法、背包問題、股市預測、投資組合,,2018,https://hdl.handle.net/11296/287vzz,本研究比較傳統時間序列與模糊時間序列預測台灣股票市場的績效，以及分析九大全球知名投資專家所採...
2,利用機器學習演算法偵測 與量化小鼠的疼痛,Detection and Quantification of Mice in Pain U...,臉部特徵識別、疼痛偵測與量化、監督式分類學習法、支援向量機、哈爾濾波器、Mask RCNN,pain face、facial pattern、detection、quantificat...,2019,https://hdl.handle.net/11296/a55786,動物試驗為現行測試藥物的重要流程之一，以止痛藥為例，動物實驗中透過將實驗動物施打藥物或手術等...
3,使用機器學習演算法進行與牙周病相關齦下微生物的總體基因體分析及特徵選擇之研究,A Study on Metagenomic Analysis and Feature Se...,總體基因體學、微生物菌落、牙周病、機器學習、特徵選擇,Metagenomics、Microbiome Community、Periodontal ...,2019,https://hdl.handle.net/11296/jhfnq7,牙周病是一種發炎性疾病，涉及口腔內微生物與宿主免疫反應之間的複雜相互作用。根據研究指出，牙周...
4,一個基於機器學習的醫療法判決書預測系統使用具文本相似性的法條分類決策樹,A Machine Learning Based Prediction System of ...,案例預測、機器學習、文本相似度、決策樹,case prediction、machine learning、word2vec、deci...,2018,https://hdl.handle.net/11296/gxyh35,"本論文提出了一種基於機器學習(Machine Learning, ML)的預測系統，該系統使..."
5,利用機器學習方法預測違約風險於不平衡P2P網路借貸數據,Using Machine Learning Techniques to Predict D...,P2P 網路借貸、合成少數類過採樣技術、類神經網路、隨機森林、自適應增強法、機器學習,P2P Lending、SMOTE、Neural Network、Random Forest...,2018,https://hdl.handle.net/11296/3x68s2,傳統金融業者辦理貸款手續繁雜、貸款週期長、貸款條件苛刻、高規格信用評等情況，讓中小企業或是個...
6,利用機器學習預測半導體製程中化學機械平坦化的晶圓材料移除率,Predicting Wafer Material Removal Rate for Sem...,化學機械平坦化、機器學習、預測模型、評分模型,Chemical mechanical polishing (CMP)、Machine le...,2018,https://hdl.handle.net/11296/8332vc,為了達到全面品質管控的目的，在半導體製程中預測晶圓的材料移除率是個很重要的一個步驟。隨著虛擬...
7,應用機器學習識別關鍵基因變異於肺癌併發間質性肺病之病患,Identifying Key Genetic Variant for Interstit...,間質性肺病、肺癌、機器學習、特徵選擇,Interstitial lung disease、Lung cancer、Machine ...,2018,https://hdl.handle.net/11296/nx75y2,肺癌長期為國人癌症死亡原因的第一位，同時世界衛生組織亦將之列為最常見的癌症死因。除了癌症之外...
8,使用機器學習之半主動式膝部輔具動態阻力控制,Dynamic Resistance Control for Semi-active Kne...,機器學習、多層式磁流變液阻力器、膝關節支架,machine learning、Magneto-rheological brake、kne...,2018,https://hdl.handle.net/11296/akkerz,我們的社會逐漸邁向高齡化，科技也不斷地隨之更新，出現許多長期照護醫療的新技術，在科技輔具的領...
9,整合機器學習之高效率影像自動化標註方法,Efficient Intelligent Automatic Image Annotati...,數據集、交通影像標註、自動標註、影像標註、機器學習,Dataset、Traffic Image Annotation、Auto Labeling...,2018,https://hdl.handle.net/11296/8787a8,結合電腦視覺的機器學習技術一直是影像辨識技術發展趨勢，從較早期的 Support Vecto...


In [571]:
# 主要爬蟲程式
def get_all_paper(from_paper_num=0,end_paper_num=10,keyword="機器學習",url="https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=9P1K1H/webmge?mode=basic"):
    browser = webdriver.Chrome('./chromedriver')
    browser.get(url)
    element=browser.find_element_by_id("ysearchinput0")
    element.send_keys(keyword, Keys.ARROW_DOWN)
    browser.find_element_by_id("gs32search").click()
    paper_page=browser.find_element_by_class_name("slink")# 選一篇進去
    paper_page.click()
    index=1
    data=list()
    for paper_number in range(from_paper_num,end_paper_num):# by each page crawler
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        data.append(get_paper_info(browser,soup))
        print("paper-"+str(paper_number+1)+" finish!")
        #browser.find_element_by_name("gonext").click()
        next_paper=browser.find_element_by_css_selector('#bodyid > form:nth-child(16) > div > table > tbody > tr:nth-child(1) > td.etds_mainct > table > tbody > tr:nth-child(6) > td > div.cont_l2 > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td:nth-child(4) > input[type="image"]')
        next_paper.click()
    browser.close()

In [572]:
get_all_paper()

TypeError: get_paper_link() missing 1 required positional argument: 'tr_len'

In [ ]:
browser = webdriver.Chrome('./chromedriver')
browser.get('https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=9P1K1H/webmge?mode=basic')
element=browser.find_element_by_id("ysearchinput0")
element.send_keys("機器學習", Keys.ARROW_DOWN)
browser.find_element_by_name("gs32search").click()
# 變化
paper_page=browser.find_element_by_class_name("slink")
paper_page.click()# 點1
data=list()
for i in range(0,2):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    link_node=soup.select('#format0_disparea > tbody')
    tr_len=len(link_node[0])
#     for tr_index in range(1,tr_len):
#         link_node=soup.select('#format0_disparea > tbody > tr:nth-of-type('+str(tr_index)+')') # 每列
        #row=link_node[0].find_all("th", class_="std1")
    print(get_paper_link(soup,tr_len))
        
        
#         if '本論文永久網址1:' in row[0].string:
#             row1=link_node[0].select("#fe_text1")
#             print(row1[0].get('value'))
#         else:
#             print('')
#         if row[0].string=='論文名稱:':
#             back=link_node[0].select("td.std2")
#             paper_chinese_title=back[0].get_text()
#             print(paper_chinese_title)
#         elif row[0].string=='論文名稱(外文):':
#             back=link_node[0].select("td.std2")
#             print(back[0].get_text())
#         elif row[0].string=='論文出版年:':
#             back=link_node[0].select("td.std2")
#             print(back[0].get_text())
#         elif row[0].string=='中文關鍵詞:':
#             back=link_node[0].select("td.std2")
#             print(back[0].get_text())
#         elif row[0].string=='外文關鍵詞:':
#             back=link_node[0].select("td.std2")
#             print(back[0].get_text())
    print("paper-"+str(i+1)+" finish!")
    next_paper=browser.find_element_by_css_selector('#bodyid > form:nth-child(16) > div > table > tbody > tr:nth-child(1) > td.etds_mainct > table > tbody > tr:nth-child(6) > td > div.cont_l2 > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td:nth-child(4) > input[type="image"]')
    next_paper.click()
browser.close()

In [466]:
import pandas as pd
pd.DataFrame(data)[['chinese_title', 'english_title', 'chinese_keyword', 'english_keyword', 'paper_year','link','abstract']].head(10)

,chinese_title,english_title,chinese_keyword,english_keyword,paper_year,link,abstract
0,利用通用型機器學習模式發展特定的機器學習範式,Development of Specific Machine Learning Parad...,人工智慧、機器學習,Artificial Intelligence、Maching Learning,1993,https://hdl.handle.net/11296/7wkt54,本論文是探討如何利用通用型機器學習模式來建構各種特定的機器學習範式。通用型模式訂出五個在機器...
1,Application of time series and machine learnin...,None,\n\n被引用:0點閱:11評分:下載:0書目收藏:0\n\n,,中文,https://hdl.handle.net/11296/287vzz,本研究比較傳統時間序列與模糊時間序列預測台灣股票市場的績效，以及分析九大全球知名投資專家所採...
2,利用機器學習演算法偵測 與量化小鼠的疼痛,Detection and Quantification of Mice in Pain U...,2019,107,電機工程研究所,https://hdl.handle.net/11296/a55786,動物試驗為現行測試藥物的重要流程之一，以止痛藥為例，動物實驗中透過將實驗動物施打藥物或手術等...
3,使用機器學習演算法進行與牙周病相關齦下微生物的總體基因體分析及特徵選擇之研究,A Study on Metagenomic Analysis and Feature Se...,2019,107,應用化學系,https://hdl.handle.net/11296/jhfnq7,牙周病是一種發炎性疾病，涉及口腔內微生物與宿主免疫反應之間的複雜相互作用。根據研究指出，牙周...
4,一個基於機器學習的醫療法判決書預測系統使用具文本相似性的法條分類決策樹,A Machine Learning Based Prediction System of ...,2018,107,資訊工程學系,https://hdl.handle.net/11296/gxyh35,"本論文提出了一種基於機器學習(Machine Learning, ML)的預測系統，該系統使..."
5,利用機器學習方法預測違約風險於不平衡P2P網路借貸數據,Using Machine Learning Techniques to Predict D...,2018,107,巨量資料管理學院碩士學位學程,https://hdl.handle.net/11296/3x68s2,傳統金融業者辦理貸款手續繁雜、貸款週期長、貸款條件苛刻、高規格信用評等情況，讓中小企業或是個...
6,利用機器學習預測半導體製程中化學機械平坦化的晶圓材料移除率,Predicting Wafer Material Removal Rate for Sem...,英文,32,工業工程學類,https://hdl.handle.net/11296/8332vc,為了達到全面品質管控的目的，在半導體製程中預測晶圓的材料移除率是個很重要的一個步驟。隨著虛擬...
7,應用機器學習識別關鍵基因變異於肺癌併發間質性肺病之病患,Identifying Key Genetic Variant for Interstit...,2018,107,工業工程與管理系所,https://hdl.handle.net/11296/nx75y2,肺癌長期為國人癌症死亡原因的第一位，同時世界衛生組織亦將之列為最常見的癌症死因。除了癌症之外...
8,使用機器學習之半主動式膝部輔具動態阻力控制,Dynamic Resistance Control for Semi-active Kne...,2018,106,車輛工程系,https://hdl.handle.net/11296/akkerz,我們的社會逐漸邁向高齡化，科技也不斷地隨之更新，出現許多長期照護醫療的新技術，在科技輔具的領...
9,整合機器學習之高效率影像自動化標註方法,Efficient Intelligent Automatic Image Annotati...,2018,106,資訊工程系,https://hdl.handle.net/11296/8787a8,結合電腦視覺的機器學習技術一直是影像辨識技術發展趨勢，從較早期的 Support Vecto...


In [441]:
data[4]

{'chinese_title': '一個基於機器學習的醫療法判決書預測系統使用具文本相似性的法條分類決策樹',
 'english_title': 'A Machine Learning Based Prediction System of Medical Laws Judgment Using Statute-Classified Decision Tree with Text Similarity',
 'chinese_keyword': '2018',
 'english_keyword': '107',
 'paper_year': '資訊工程學系',
 'link': 'https://hdl.handle.net/11296/gxyh35',
 'abstract': '本論文提出了一種基於機器學習(Machine Learning, ML)的預測系統，該系統使用具有文本相似性(Text Similarity, TS)的法條分類決策樹(Decision Tree, DT)，通過分析法條和法律語料，系統預測訴訟結果屬於原告勝訴還是被告勝訴。該系統的應用領域是協助律師。該系統節省了律師的文本檢索時間並提供了高準確度的預測。預測系統的首要任務是分析判決書的法條。當法條不能做出準確的預測時，進階考慮案例文本之間的相似性，並採用文本相似性(Text Similarity, TS)的方法。'}

In [ ]:
#format0_disparea > tbody